# Workshop 2 - Manuela Mayorga Rojas

#### This workshop is about how to build an ETL pipeline using Apache Airflow, this will be done using two data sources (csv file, database), the first data set is from Spotify (csv file) and the second set is from Grammys (database), using apache airflow we will read the data, one as a csv and another using SQLAlquemys database engine, also using Airflow we will perform transformations, then, merge both data sets and load them in google drive, finally visualize this information in PowerBI. 
 
### Objectives:

- Use Apache Airflow to read data from multiple sources, such as CSV files and databases.
- Apply transformations to the data read using Apache Airflow.
- upload the transformed data to an external storage platform (Google Drive)
- Merge data sets 
- Use PowerBI to visualize information


This part covers operations related to data management, creation and configuration of tables in the database. From database connection and data manipulation to table creation and data loading. This process is performed to load the initial dataset of the grammys. 

The environment variables are loaded from a `.env` Processor. The main goal is to manage the configuration, and the `WORK_PATH` environment variable is used to define a working directory.

In [6]:
import sys
import os
from dotenv import load_dotenv

load_dotenv()
work_path = os.getenv('WORK_PATH')

sys.path.append(work_path)


The necessary libraries are imported for use

In [7]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import inspect, Table, MetaData, insert
from src.database.connection import config_loader
from src.data_process.class_processor import Processor
from src.models.models import Grammys
import logging


In [3]:
connection = config_loader()

Session = sessionmaker(bind=config_loader)
session = Session()

INFO:root:Connected successfully to Workshop-2 database


A connection to a database is established using `connection` and a SQLAlchemy session is created using `sessionmaker` linked to that connection. This streamlines database operations in the program.

In [5]:
try:
    Grammys.__table__.create(connection)
    print("Table created successfully.")
except Exception as e:
    print(f"Error creating table: {e}")

Table created successfully.


Load the data in the 'grammys' table, and in case of an error, indicate which one it is.

In [8]:
data = Processor('../data/the_grammy_awards.csv')
data.insert_id()
df = data.df

try:
    metadata = MetaData()
    table = Table('grammys', metadata, autoload=True, autoload_with=connection)

    with connection.connect() as conn:
        values = [{col: row[col] for col in df.columns} for _, row in df.iterrows()]

        conn.execute(insert(table), values)
            
        logging.info("Data inserted successfully.")
except Exception as e:
        logging.error(f"Error inserting data: {e}")


/tmp/ipykernel_5484/774297178.py:7: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  table = Table('grammys', metadata, autoload=True, autoload_with=connection)
INFO:root:Data inserted successfully.


# References
The following links served as a guide to run this work with the necessary tools. 


1. GroverTec. (2024, 24 febrero). Como Instalar Python en Linux Ubuntu: Guía Sencilla para Principiantes [Vídeo]. YouTube. https://www.youtube.com/watch?v=88np4KkfDO8 

2. Enreta Services. (2022, 22 abril). 🟣 Cómo instalar UBUNTU 22.04 PASO a PASO desde cero! TUTORIAL 📀 [Vídeo]. YouTube. https://www.youtube.com/watch?v=8MRibUo9VAA  

3. kipuna ec. (2024, 25 febrero). Instalar git en Ubuntu [Vídeo]. YouTube. https://www.youtube.com/watch?v=4M8cL-lXANQ  

4. Roelcode. (2022, 13 septiembre). 🐘Instalar PostgreSQL 14 y PgAdmin4 en Linux 🐧 Ubuntu 22.04 y distros basados en Ubuntu [Vídeo]. YouTube. https://www.youtube.com/watch?v=5sP36Hdh4wU  